In [57]:
import glob
import pandas as pd
import os
import json

In [65]:
inputdir = "../../data/website/databricks/hourly"
outputdir = "../../data/website/test/hourly"

problematicpaths = []

for tt in ["user", "spider"]:
    for at in ["desktop", "mobile-dev", "mobile-app"]:
        for domain in ["en.wikipedia", "de.wikipedia", "fr.wikipedia", "es.wikipedia", "ru.wikipedia", "zh.wikipedia"]:
            for year in [2019]:
                for month in [9]:
                    for day in range(1,32):

                        jsondic = {i:[] for i in range(24)}
                        filepathpattern = f"./{inputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}-*.csv"
                        filepaths = glob.glob(filepathpattern)
                        
                        if not filepaths:
                            print(f"{filepathpattern} IS EMPTY")
                            problematicpaths.append(filepathpattern)
                            continue

                        elif len(filepaths) != 24:
                            print(f"{filepathpattern} DOES NOT CONTAIN 24 HOURS")
                            problematicpaths.append(filepathpattern)
                            continue
                        
                        for filepath in filepaths:
                            hour = int(filepath.split("-")[-1].split(".")[0])
                            nested_csvs = glob.glob(filepath + "/*.csv")
                            if len(nested_csvs) != 1:
                                print(f"{filepathpattern} CONTAINS NOT EXACTLY 1 NESTED CSV")
                                break
                            realfilepath = nested_csvs[0]
                            with open(realfilepath, "r") as f:
                                for line in f.read().split("\n"):
                                    x, y = line.split(",")
                                    jsondic[hour].append({"x":int(x), "y":int(y)})

                        outfilepath = f"./{outputdir}/{tt}/{at}/{domain}/{year}-{month:02d}-{day:02d}.json"
                        outfiledir = "/".join(outfilepath.split("/")[:-1])

                        if not os.path.exists(outfiledir):
                            os.makedirs(outfiledir)

                        with open(outfilepath, "w") as f:
                            json.dump(jsondic, f)

# Monthly Data

In [71]:
# !dbfs cp -r "dbfs:/mnt/group09/websitedatamonthly/sep2019.parquet" "../../data/website/databricks/monthly/sep2019.parquet"

In [74]:
df = pd.read_parquet("../../data/website/databricks/monthly/sep2019.parquet")
df["date"] = pd.to_datetime(df["timestamp"], unit="s")

In [78]:
df

,timestamp,domain,trafficType,accessType,sumcount,date
0,1569204000,es.wikipedia,user,mobile-web,59591,2019-09-23 02:00:00
1,1567548000,zh.wikipedia,spider,mobile-web,1652,2019-09-03 22:00:00
2,1568260800,zh.wikipedia,user,mobile-app,21,2019-09-12 04:00:00
3,1568653200,en.wikipedia,spider,desktop,208181,2019-09-16 17:00:00
4,1569326400,es.wikipedia,spider,mobile-web,814,2019-09-24 12:00:00
...,...,...,...,...,...,...
21564,1569578400,en.wikipedia,spider,mobile-web,24723,2019-09-27 10:00:00
21565,1567720800,ru.wikipedia,spider,mobile-web,658,2019-09-05 22:00:00
21566,1568343600,en.wikipedia,user,mobile-web,50301,2019-09-13 03:00:00
21567,1569117600,es.wikipedia,user,desktop,6884,2019-09-22 02:00:00


In [87]:
outputdir = "../../data/website/test/monthly"

problematicpaths = []

for tt in ["user", "spider"]:
    for at in ["desktop", "mobile-dev", "mobile-app"]:
        for domain in ["en.wikipedia", "de.wikipedia", "fr.wikipedia", "es.wikipedia", "ru.wikipedia", "zh.wikipedia"]:
            for year in [2019]:
                for month in [9]:

                    jsondic = []

                    filtered_df = df[
                        (df["domain"] == domain) &
                        (df["accessType"] == at) &
                        (df["trafficType"] == tt) &
                        (df["date"].dt.year == year) &
                        (df["date"].dt.month == month)
                    ]

                    filtered_df.sort_values("timestamp", ascending=True, inplace=True)

                    for _, (date, y) in filtered_df[["date", "sumcount"]].iterrows():
                        x = f"{date.year:04d}-{date.month:02d}-{date.day:02d}-{date.hour:02d}"
                        jsondic.append({"x":x, "y":int(y)})

                    outfilepath = f"./{outputdir}/{tt}/{at}/{domain}/{year}-{month:02d}.json"
                    outfiledir = "/".join(outfilepath.split("/")[:-1])  # the directory where the file is in

                    if not os.path.exists(outfiledir):
                        os.makedirs(outfiledir)

                    with open(outfilepath, "w") as f:
                        json.dump(jsondic, f)